In [ ]:
import pandas as pd
from databaker.framework import *
from databakerUtils.writers import v4Writer
from databakerUtils.v4Functions import v4Integers
import requests

#choose year
inputTime = '2017'

inputFile = 'ukbusinessworkbook2017.xls'
outputFile1 = 'v4_uk_business_industry_' + inputTime + '.csv'
outputFile2 = 'v4_uk_business_sic_' + inputTime + '.csv'

allTabs = loadxlstabs(inputFile)

#Broad Industry Group
def industryLookup(value):
    lookup = {
            '01-03' : '01-03 : Agriculture, forestry & fishing',
            '05-39' : '05-39 : Production',
            '41-43' : '41-43 : Construction',
            '45' : '45 : Motor trades',
            '46' : '46 : Wholesale',
            '47' : '47 : Retail',
            '49-53' : '49-53 : Transport & Storage (inc. postal)',
            '55-56' : '55-56 : Accommodation & food services',
            '58-63' : '58-63 : Information & communication',
            '64-66' : '64-66 : Finance & insurance',
            '68' : '68 : Property',
            '69-75' : '69-75 : Professional, scientific & technical',
            '77-82' : '77-82 : Business administration & support services',
            '84' : '84 : Public administration & defence',
            '85' : '85 : Education',
            '86-88' : '86-88 : Health',
            '90-99' : '90-99 : Arts, entertainment, recreation & other',
            'Total':'Total for Broad Industry Groups'
            }
    return lookup[value]

def tabNameLookup(value):
    lookup = {
            'Table 1':'Enterprises',
            'Table 16':'Local Units',
            'Table 4':'Enterprises',
            'Table 19':'Local Units'
            }
    return lookup[value]

#import labels from codelist api
url = 'https://api.cmd-dev.onsdigital.co.uk/v1/code-lists/admin-geography/editions/one-off/codes'
r = requests.get(url)
wholeDict = r.json()
geogDict = {}
for item in wholeDict['items']:
    geogDict.update({item['id']:item['label']})

#function to use correct labels
def geogLookup(value):
    return geogDict[value]

#sic code list
url = 'https://api.beta.ons.gov.uk/v1/code-lists/sic/editions/one-off/codes'
r = requests.get(url)
wholeDict = r.json()
sicDict = {}
for item in wholeDict['items']:
    sicDict.update({item['id']:item['label']})

def SICLookup(value):
    return sicDict[value]

def SicCodesFormatter(value):
    if len(value) == 4:
        newValue = value[:2] + '-' + value[2:]
    else:
        newValue = value
    return newValue

#data from these tabs - each group has same format  
'''Will need to change'''
Group1 = ['Table 1','Table 16']
Group2 = ['Table 4','Table 19']

tabsGroup1 = [tab for tab in allTabs if tab.name in Group1]
tabsGroup2 = [tab for tab in allTabs if tab.name in Group2]


'''DataBaking'''

#first loop for first format
conversionsegments = []

for tab in tabsGroup1:
    
    geographyCodes = tab.excel_ref('A7').expand(DOWN).is_not_blank().is_not_whitespace()
    geographyLabels = tab.excel_ref('B7').expand(DOWN).is_not_blank().is_not_whitespace()
    
    industry = tab.excel_ref('C6').expand(RIGHT).is_not_blank().is_not_whitespace()
    
    tabName = tab.excel_ref('C1').value.split('-')[0].strip()
    
    obs = geographyCodes.waffle(industry).is_not_blank().is_not_whitespace()
    
    dimensions = [
            HDimConst(TIME, inputTime),
            HDim(geographyCodes, GEOG, DIRECTLY, LEFT),
            HDim(geographyLabels, 'geogLabels', DIRECTLY, LEFT),
            HDim(industry, 'industry', DIRECTLY, ABOVE),
            HDimConst('tabName', tabName)
            ]
    
    conversionsegment = ConversionSegment(tab, dimensions, obs).topandas()
    conversionsegments.append(conversionsegment)
   
data1 = pd.concat(conversionsegments)
df1 = v4Writer(outputFile1, data1, asFrame = True)

#second loop for second format
conversionsegments = []

for tab in tabsGroup2:
    
    sic = tab.excel_ref('A8').expand(DOWN).is_not_blank().is_not_whitespace()
    
    employment = tab.excel_ref('B7').expand(RIGHT).is_not_blank().is_not_whitespace()
    
    tabName = tab.excel_ref('B1').value.split('-')[0].strip()
    
    obs = sic.waffle(employment).is_not_blank().is_not_whitespace()
    
    dimensions = [
            HDimConst(TIME, inputTime),
            HDimConst(GEOG, 'geog'),
            HDim(sic,'SIC', DIRECTLY, LEFT),
            HDim(employment, 'employment', DIRECTLY, ABOVE),
            HDimConst('tabName', tabName)
            ]
    
    conversionsegment = ConversionSegment(tab, dimensions, obs).topandas()
    conversionsegments.append(conversionsegment)
    
data2 = pd.concat(conversionsegments)
df2 = v4Writer(outputFile2, data2, asFrame = True)

'''Post Processing'''

#columns to be renamed
renameCols = {
        'Geography':'geography',
        'Geography_codelist':'admin-geography',
        'industry_codelist':'uk-business-broad-industry-group',
        'industry':'broadindustrygroup',
        'tabName_codelist':'uk-business-unit',
        'tabName':'unit',
        'SIC_codelist':'sic',
        'SIC':'standardindustrialclassification',
        'employment':'employmentsizeband',
        'employment_codelist':'uk-business-employment-size-band',
        'Time_codelist':'calendar-years',
        'Time':'time'
        }

df1['Geography'] = df1['Geography_codelist'].apply(geogLookup)

df1 = df1.drop(['geogLabels_codelist', 'geogLabels'],axis=1)

df1['Time_codelist'] = df1['Time']

df1['industry_codelist'] = df1['industry'].apply(lambda x:x.split(':')[0].strip())
df1['industry'] = df1['industry_codelist'].apply(industryLookup)
df1['industry_codelist'] = df1['industry_codelist'].apply(lambda x:x.lower())

df1['tabName'] = df1['tabName'].apply(tabNameLookup)
df1['tabName_codelist'] = df1['tabName'].apply(lambda x:x.lower().replace(' ','-'))

df1 = df1.rename(columns = renameCols)
#reorder
df1 = df1[['V4_0', 'calendar-years', 'time', 'admin-geography', 'geography',
       'uk-business-unit', 'unit', 'uk-business-broad-industry-group', 'broadindustrygroup']]


df2 = df2.drop(['Geography_codelist', 'Geography'], axis=1)

df2['Time_codelist'] = df2['Time']

df2['SIC_codelist'] = df2['SIC'].apply(lambda x:x.split(':')[0].strip().lower())
df2['SIC_codelist'] = df2['SIC_codelist'].apply(SicCodesFormatter)
df2['SIC'] = df2['SIC_codelist'].apply(SICLookup)

df2['employment_codelist'] = df2['employment'].apply(lambda x:x.lower())

df2['tabName'] = df2['tabName'].apply(tabNameLookup)
df2['tabName_codelist'] = df2['tabName'].apply(lambda x:x.lower().replace(' ','-'))

df2 = df2.rename(columns = renameCols)

df1['V4_0'] = df1['V4_0'].apply(v4Integers)
df2['V4_0'] = df2['V4_0'].apply(v4Integers)

df1.to_csv(outputFile1, index = False)
#df2.to_csv(outputFile2, index = False)